In [1]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
from random import randint

##Создание датасета с синтезированными текстами


In [ ]:
Ai_texts1 = pd.read_csv('Ai_texts.csv')

In [ ]:
Ai_text = []

In [ ]:
for s in Ai_texts1['Текст']:
  s = s.replace("\n", "")
  Ai_text.append(s)

In [ ]:
Ai_texts2 = pd.read_csv('AI_Texts2.csv')

In [ ]:
for s in Ai_texts2['Текст']:
  s = s.replace("\n", "")
  Ai_text.append(s)

In [ ]:
len(Ai_text)

14506

In [ ]:
Zero = []
for i in range(len(Ai_text)):
  Zero.append(0)

In [ ]:
df1 = pd.DataFrame({"Текст":Ai_text, "Автор":Zero})

##Создание датасета с естественными текстами

In [ ]:
def check(t):
    t = str.lower(t)
    count = 0
    bad_lst = ["\\", "//", "/", "@", "\\n", "[", "]", "(", ")", "a", "i", "e"]
    if len(t) >= 100:
        count += 1
    for i in bad_lst:
        if i not in t:
            count += 1
    if count == len(bad_lst) + 1:
        return True
    else:
        return False

In [ ]:
file = open('All_texts.txt', "r")

In [ ]:
texts = []

In [ ]:
for line in file:
  line = line.replace(u'\xa0',' ')
  line = line.replace(u"\n", "")
  texts.append(line)

In [ ]:
len(texts)

201271

In [ ]:
human = []
while len(human) != 25000:
    n = randint(0, 201270)
    s = texts[n]
    if check(s):
      human.append(s)

In [ ]:
ones = []
for _ in range(len(human)):
  ones.append(1)

In [ ]:
df2 = pd.DataFrame({'Текст':human, 'Автор':ones})

In [ ]:
data = pd.concat([df1, df2], axis = 0)
data = data.sample(frac=1)
data

,Текст,Автор
13677,"синильники, которые, как и все остальные, были...",0
4117,"Господа взъерепенились, еще больше сулят, а су...",1
16940,"– Ну, в таком разе и прекрасно, а то я думал, ...",1
6903,Еще история про коров. Когда Нью-Йорк расширял...,1
11586,"естествоведы, археологи, историки, философы, п...",0
...,...,...
14268,"подматывание» к нему.– А что, если я не смогу?...",0
330,"– Точно, так и было все! – подтвердил Вася. – ...",1
20198,Издатели и редакторы заваливали его письмами: ...,1
19023,8. Секта всегда стремится расширить свои круги...,1


##Токенизация

In [3]:
num_words = 999999999
max_len = 100

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data['Текст'])

In [ ]:
x_train = data['Текст'][0:35000]
x_test = data['Текст'][35001:39505]
y_train = data['Автор'][0:35000]
y_test = data['Автор'][35001:39505]

In [ ]:
seq_train = tokenizer.texts_to_sequences(x_train)
seq_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
x_train = pad_sequences(seq_train, maxlen=max_len)
x_test = pad_sequences(seq_test, maxlen=max_len)

##Создание модели

In [ ]:
def build_LSTM(len, boxes, optimizer):
  model = Sequential()
  model.add(Embedding(num_words, len, input_length=max_len))
  model.add(LSTM(boxes))
  model.add(Dense(1, "sigmoid"))
  model.compile(optimizer=optimizer, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  return model

In [ ]:
LSTM = Sequential()
LSTM.add(Embedding(num_words, 32, input_length=max_len))
LSTM.add(LSTM(64))
LSTM.add(Dense(1, "sigmoid"))
LSTM.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy'])